<a href="https://colab.research.google.com/github/rmcnew/Pneumonia_Classifier/blob/master/Pneumonia_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!which gdown

/usr/local/bin/gdown


In [20]:
# Richard Scott McNew
# A02077329
# CS 6600: Intelligent Systems

from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import drive, files
import io
import os
import pathlib
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# enable accelerated linear algebra
tf.config.optimizer.set_jit(True)
# enable tensorflow AUTOTUNE
AUTOTUNE = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 8
IMAGE_SIDE = 450 
SHUFFLE_SIZE = 25
EPOCHS = 2


DATASET_PATH = "/content/Pneumonia_Classifier/dataset"

def get_dataset_files():
    if not os.path.isdir(DATASET_PATH):
        print("Downloading the dataset . . .")
        !git clone -b dataset_only https://github.com/rmcnew/Pneumonia_Classifier.git 
    else:
        print("Using previously downloaded dataset")


# dataset paths
get_dataset_files()
dataset = pathlib.Path(DATASET_PATH)
test = dataset.joinpath("test")
test_count = len(list(test.glob('**/*.jpeg')))
train = dataset.joinpath("train")
train_count = len(list(train.glob('**/*.jpeg')))
validate = dataset.joinpath("validate")
validate_count = len(list(validate.glob('**/*.jpeg')))

####################### Dataset Preprocessing #########################
def create_train_image_generator():
    train_image_generator = ImageDataGenerator(rescale=1./255, zoom_range=0.5)
    train_data_gen = train_image_generator.flow_from_directory(
            batch_size=BATCH_SIZE, 
            directory=str(train), 
            shuffle=True, 
            target_size=(IMAGE_SIDE, IMAGE_SIDE), 
            class_mode='binary')
    return train_data_gen

def create_test_image_generator():
    test_image_generator = ImageDataGenerator(rescale=1./255)
    test_data_gen = test_image_generator.flow_from_directory(
            batch_size=BATCH_SIZE, 
            directory=str(test), 
            target_size=(IMAGE_SIDE, IMAGE_SIDE), 
            class_mode='binary')
    return test_data_gen

def create_validate_image_generator():
    validate_image_generator = ImageDataGenerator(rescale=1./255)
    validate_data_gen = validate_image_generator.flow_from_directory(
            batch_size=BATCH_SIZE, 
            directory=str(validate), 
            target_size=(IMAGE_SIDE, IMAGE_SIDE), 
            class_mode='binary')
    return validate_data_gen

############################ Model Creation and Loading ##############################

# val 90
#def create_model():
#    model = Sequential([
#        Conv2D(450, 10, padding='same', activation='relu', kernel_regularizer='l2', input_shape=(450, 450, 3)),
#        MaxPooling2D(),
#        Conv2D(225, 5, padding='same', activation='relu', kernel_regularizer='l2'),
#        MaxPooling2D(),
#        Dropout(0.2),
#        Conv2D(100, 5, padding='same', activation='relu', kernel_regularizer='l2'),
#        MaxPooling2D(),        
#        Flatten(),
#        Dense(512, activation='relu'),
#        Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy']) 
#    return model


def create_model():
    model = Sequential([
        Conv2D(IMAGE_SIDE, 10, padding='same', activation='relu', kernel_regularizer='l2', input_shape=(IMAGE_SIDE, IMAGE_SIDE, 3)),
        MaxPooling2D(),
        Conv2D(225, 5, padding='same', activation='relu', kernel_regularizer='l2'),
        MaxPooling2D(),
        Dropout(0.2),
        Conv2D(100, 5, padding='same', activation='relu', kernel_regularizer='l2'),
        MaxPooling2D(),        
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy']) 
    return model

    
# There is a trained persisted model that is shared from my Google Drive account
# Download a copy to local storage for use in evaluating against the datasets
def download_trained_model_from_google_drive_shared():
    print("Downloading trained model from Google Drive . . .")
    !gdown https://drive.google.com/uc?id=1Qwm5QlveZsUzO6vU7jMBUOgx_uHlSN9i
    print("Download completed!  Loading the model . . .")
    model = tf.keras.models.load_model("/content/pneumonia_classifier_model", custom_objects=None, compile=True)
    print("Model loaded.")
    return model


########################### Google Drive functions #############################
# Note that these functions will require interactive steps to create OAuth 
# tokens that will be used to authorize access to Google Drive

# Google Drive path to the saved model
MODEL_PATH = "/content/drive/My Drive/USU/intelligent_systems/Pneumonia_Classifier/pneumonia_classifier_model"

# mount Google drive
def mount_drive():
    drive.mount('/content/drive')

# unmount Google drive
def unmount_drive():
    drive.flush_and_unmount()

def save_model_to_google_drive(model):
    mount_drive()
    model.save(MODEL_PATH)
    unmount_drive()

def load_trained_model_from_google_drive():
    mount_drive()
    model = tf.keras.models.load_model(MODEL_PATH, custom_objects=None, compile=True)
    unmount_drive()
    return model


###################### Model Training and Evaluation ##############################
def train_model():
    train_data_gen = create_train_image_generator()
    test_data_gen = create_test_image_generator()
    model = create_model()
    model.fit(
        train_data_gen,        
        steps_per_epoch=train_count // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_data_gen,        
        validation_steps=test_count // BATCH_SIZE
    ) 
    save_model_to_google_drive(model)


def train_model_more():
    train_data_gen = create_train_image_generator()
    test_data_gen = create_test_image_generator()
    model = load_trained_model_from_google_drive()
    model.fit(
        train_data_gen,
        steps_per_epoch=train_count // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_data_gen,
        validation_steps=test_count // BATCH_SIZE
    )    
    save_model_to_google_drive(model)
    
def evaluate_trained_model():
    test_data_gen = create_test_image_generator()
    model = download_trained_model_from_google_drive_shared()
    model.evaluate(test_data_gen)


evaluate_trained_model()
#train_model_more()
#unmount_drive()

Using previously downloaded dataset
Found 624 images belonging to 2 classes.
Downloading...
From: https://drive.google.com/uc?id=1Qwm5QlveZsUzO6vU7jMBUOgx_uHlSN9i
To: /content/pneumonia_classifier_model
655MB [00:05, 116MB/s]
Download completed!  Loading the model . . .
Model loaded.
78/78 [==============================] - 18s 236ms/step - loss: 0.3489 - acc: 0.8846
